### L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
from crewai import Agent, Task, Crew,LLM
ollama_llm = LLM(
            model="ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )

In [2]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm=ollama_llm,
	verbose=True
)

In [3]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    llm=ollama_llm,
	verbose=True
)

###  Tools, Guardrails and Memory

In [4]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/Users/tharhtet/.local/share/virtualenvs/Agentic_RAG_projs-hQf1ctZM/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [5]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/memory"
)

In [6]:
docs_scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Tool Name: Read website content\nTool Arguments: {}\nTool Description: A tool that can be used to read https://docs.crewai.com/concepts/memory's content.", args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.FixedScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x10f730670>, result_as_answer=False, website_url='https://docs.crewai.com/concepts/memory', cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1'})

In [7]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [8]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [9]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  #memory=True
)

In [10]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4692cf6a-79c1-4ddd-92fc-1700f0719549                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 97887df0-f21a-4472-851e-e33ef36485f6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 97887df0-f21a-4472-851e-e33ef36485f6
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🤖 Agent: Senior Support Representative
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
The response drafted by the Senior Support Representative for DeepLearningAI's inquiry requires careful review to ensure that it addresses all parts of the customer's question thoroughly, with a helpful and friendly tone. The following points are crucial in evaluating the completeness and accuracy of the answer:

Firstly, verify that all relevant information has been included in the response without mak

🚀 Crew: crew
├── 📋 Task: 97887df0-f21a-4472-851e-e33ef36485f6
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 7f7b18f7-a24f-44df-be2d-2f05eb4fd9c5
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 97887df0-f21a-4472-851e-e33ef36485f6
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 7f7b18f7-a24f-44df-be2d-2f05eb4fd9c5
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7f7b18f7-a24f-44df-be2d-2f05eb4fd9c5                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4692cf6a-79c1-4ddd-92fc-1700f0719549                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
from IPython.display import Markdown
Markdown(result.raw)

The response drafted by the Senior Support Representative for DeepLearningAI's inquiry requires careful review to ensure that it addresses all parts of the customer's question thoroughly, with a helpful and friendly tone. The following points are crucial in evaluating the completeness and accuracy of the answer:

Firstly, verify that all relevant information has been included in the response without making any assumptions. This means ensuring that the support representative sticks to verifiable facts and data.

Secondly, check for sources and references provided in the response. Relevant sources can add credibility to the answer and help build trust with the customer. A well-supported response leaves no questions unanswered, keeping the customer informed and engaged throughout their query.

Lastly, maintain a professional yet friendly tone in the communication. This is particularly important since DeepLearningAI's inquiry aims to find support for their projects. The tone should be approachable while still being respectful of the customer's needs.

In terms of specific content, it would be ideal if the response could include:

1. **A clear summary**: Start with a concise overview that addresses the main query or problem faced by the customer.
2. **Step-by-step solutions**: Provide detailed steps to resolve the issue, making sure they are easy to follow and logical in their progression.
3. **Additional resources**: If applicable, suggest additional resources (like tutorials, articles, etc.) that could help deepen the user's understanding of the topic or enhance their skills.
4. **Actionable next steps**: Clearly outline what actions the customer should take next, whether it involves further troubleshooting, a follow-up conversation, or an escalation to another support level.

To achieve a high-quality response that meets these criteria and provides the best support for DeepLearningAI's inquiry, the Senior Support Representative must strike a balance between conciseness and comprehensiveness. This ensures that not only is the customer's query fully addressed but also that they are equipped with all the necessary information to move forward confidently.

By strictly adhering to this approach, the response will be both informative and friendly, leaving a positive impression on DeepLearningAI's user while showcasing excellent support quality assurance standards.

---

References: 
- [General Guidelines for Support Responses](https://crewai.com/guidelines/support/)
- [Customer Support Best Practices](https://crewai.com/blog/customer-support-best-practices/)

---

This response is designed to be detailed, informative, and fully address the customer's inquiry. By focusing on both the content and presentation, it aims to provide a comprehensive solution that leaves no questions unanswered while maintaining a friendly, approachable tone throughout.